In [1]:
import shutil
import tensorflow as tf

/home/thomas/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
CSV_COLUMNS = ['Sex',
               'Length',
               'Diameter',
               'Height',
               'Whole_weight',
               'Shucked_weight',
               'Viscera_weight',
               'Shell_weight',
               'Rings']
LABEL_COLUMN = 'Rings'
DEFAULTS = [['M'], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [1]]

def read_dataset(filename, mode, batch_size = 200):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        filenames_dataset = tf.data.Dataset.list_files(filename)
        # Read lines from text files
        textlines_dataset = filenames_dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = textlines_dataset.map(decode_csv)
    
        # Note:
        # use tf.data.Dataset.flat_map to apply one to many transformations (here: filename -> text lines)
        # use tf.data.Dataset.map      to apply one to one  transformations (here: text line -> feature list)
    
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # loop indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
    
        return dataset.make_one_shot_iterator().get_next()
    return _input_fn
    

def get_train():
    return read_dataset('./abalone/train_abalone.csv', mode = tf.estimator.ModeKeys.TRAIN)
def get_test():
    return read_dataset('./abalone/test_abalone.csv.csv', mode = tf.estimator.ModeKeys.EVAL)

In [3]:
def create_feature_cols():
    return [
        tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list('Sex', ['M','F', 'I'])),
        tf.feature_column.numeric_column('Length'),
        tf.feature_column.numeric_column('Diameter'),
        tf.feature_column.numeric_column('Height'),
        tf.feature_column.numeric_column('Whole_weight'),
        tf.feature_column.numeric_column('Shucked_weight'),
        tf.feature_column.numeric_column('Viscera_weight'),
        tf.feature_column.numeric_column('Shell_weight'),  
    ]

In [4]:

def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.DNNRegressor(hidden_units=[10,10],model_dir=output_dir,
                                         feature_columns=create_feature_cols())
    train_spec = tf.estimator.TrainSpec(input_fn=get_train(), max_steps=num_train_steps)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_test(),
                                     steps=None,
                                     start_delay_secs=1,
                                     throttle_secs=5)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [5]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = './abalone_trained'
shutil.rmtree(OUTDIR, ignore_errors=True) # start fresh each time
train_and_evaluate(OUTDIR, 1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './abalone_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fced0e5f160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 5 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSa